Nesse material a gente vai lembrar das formas de normalização de um arquivo JSON transformando esse arquivo em um DataFrame manipulável.

> ***Nota:*** *Todos os dados utilizados nos exemplos desse notebook são fictícios.*

Quando trabalhamos com arquivos JSON, é importante saber como os dados são dispostos em arquivos como esse:
- Arquivos JSON armazenam dados em pares chave-valor (formato de dicionário) ou listas.
- No `arquivo_1.json`, temos uma lista em que cada elemento é um dicionário que se refere a uma linha do conjunto de dados, mostrando as colunas e os dados.

Vamos importar esse arquivo e transformá-lo em um dataframe!

Utilizaremos a biblioteca [`pandas`](https://pandas.pydata.org/docs/) para ler o arquivo JSON.

In [2]:
import pandas as pd

Lemos o arquivo JSON com a funcionalidade [`pd.read_json`](https://pandas.pydata.org/docs/reference/api/pandas.read_json.html)

In [4]:
df = pd.read_json('Dados/arquivo_1.json')
df

,id_vendedor,id_cliente,produto,quantidade,valor_unitario,valor_total
0,101,1001,Notebook,1,3500,3500
1,102,1002,Smartphone,2,1200,2400
2,103,1003,Monitor,3,800,2400


Então conseguimos ter o resultado final.

Arquivos JSON, dependendo de como forem criados, podem ter informações aninhadas em níveis. Essa estrutura pode ser observada no `arquivo_2.json`.
- No `arquivo_2` temos a situação de um JSON aninhado com dois níveis de chaves.
- A função `pd.read_json` não vai transformar corretamente esse arquivo em um dataframe por que essa função só considera o primeiro nível de uma arquivo JSON.

In [5]:
df = pd.read_json('Dados/arquivo_2.json')
df

,id_vendedor,id_cliente,detalhes_compra
0,101,1001,"{'produto': 'Notebook', 'quantidade': 1, 'valo..."
1,102,1002,"{'produto': 'Smartphone', 'quantidade': 2, 'va..."
2,103,1003,"{'produto': 'Monitor', 'quantidade': 3, 'valor..."


Para ter o dataframe ajustado corretamente, devemos aplicar a normalização.

Para aplicar a normalização, usamos a função [`pd.json_normalize`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html) do pandas.

Mas antes de ir direto para a aplicação dela, aqui vem uma **dica**:
- Aplicar a normalização nesses casos de aninhamento pode ser mais trabalhoso se usarmos um dataframe, pois essa função recebe como argumento uma lista de dicionários, ou um dicionário, não um DataFrame Pandas.
- Se quisermos muito usar um dataframe, teríamos que passar a coluna onde está acontecendo o problema e ainda concatenar a coluna no dataframe original.

Então, seguiremos o modo mais tranquilo, que é fazer a leitura desse arquivo JSON e armazená-lo como uma lista Python.

Usamos a biblioteca [`json`](https://docs.python.org/3/library/json.html#) para manipular arquivos JSON.

Para ler o arquivo JSON, usamos a função [`json.load`](https://docs.python.org/3/library/json.html#json.load)

In [6]:
import json

In [8]:
with open('Dados/arquivo_2.json', 'r') as arquivo:
    dados = json.load(arquivo)
dados

[{'id_vendedor': 101,
  'id_cliente': 1001,
  'detalhes_compra': {'produto': 'Notebook',
   'quantidade': 1,
   'valor_unitario': 3500.0,
   'valor_total': 3500.0}},
 {'id_vendedor': 102,
  'id_cliente': 1002,
  'detalhes_compra': {'produto': 'Smartphone',
   'quantidade': 2,
   'valor_unitario': 1200.0,
   'valor_total': 2400.0}},
 {'id_vendedor': 103,
  'id_cliente': 1003,
  'detalhes_compra': {'produto': 'Monitor',
   'quantidade': 3,
   'valor_unitario': 800.0,
   'valor_total': 2400.0}}]

Para transformar isso em um DataFrame, basta enviar `dados` à função [`json_normalize`](https://pandas.pydata.org/docs/reference/api/pandas.json_normalize.html).

In [9]:
df = pd.json_normalize(dados)
df

,id_vendedor,id_cliente,detalhes_compra.produto,detalhes_compra.quantidade,detalhes_compra.valor_unitario,detalhes_compra.valor_total
0,101,1001,Notebook,1,3500.0,3500.0
1,102,1002,Smartphone,2,1200.0,2400.0
2,103,1003,Monitor,3,800.0,2400.0


As colunas aninhadas em `detalhes_compra` recebem o prefixo do nome da coluna `detalhes_compra` seguido de um `.`. Isso é o padrão da função, mas pode ser alterado através do parâmetro `sep`, conforme informado na documentação. Exemplo:

```python
df = pd.json_normalize(dados, sep='_')
df
```

Na **saída**, percebemos que o separador das colunas novas é o `'_'`:

|    |   id_vendedor |   id_cliente | detalhes_compra_produto   |   detalhes_compra_quantidade |   detalhes_compra_valor_unitario |   detalhes_compra_valor_total |
|---:|--------------:|-------------:|:--------------------------|-----------------------------:|---------------------------------:|------------------------------:|
|  0 |           101 |         1001 | Notebook                  |                            1 |                             3500 |                          3500 |
|  1 |           102 |         1002 | Smartphone                |                            2 |                             1200 |                          2400 |
|  2 |           103 |         1003 | Monitor                   |                            3 |

Um último detalhe sobre a normalização de arquivos JSON é o comportamento da função `pd.json_normalize` quando se depara com uma estrutura que contém listas, como a mostrada abaixo:

In [10]:
dados = [
            {
                "id_vendedor": 101,
                "id_cliente": 1001,
                "compras": [
                    {"produto": "Notebook", "quantidade": 1, "valor_unitario": 3500.00, "valor_total": 3500.00},
                    {"produto": "Mouse", "quantidade": 2, "valor_unitario": 50.00, "valor_total": 100.00}
                ]
            },
            {
                "id_vendedor": 102,
                "id_cliente": 1002,
                "compras": [
                    {"produto": "Smartphone", "quantidade": 1, "valor_unitario": 1200.00, "valor_total": 1200.00},
                    {"produto": "Fone de Ouvido", "quantidade": 1, "valor_unitario": 200.00, "valor_total": 200.00}
                ]
            }
        ]

Se tentarmos transformar esses dados diretamente em um DataFrame:

In [11]:
df = pd.json_normalize(dados)
df

,id_vendedor,id_cliente,compras
0,101,1001,"[{'produto': 'Notebook', 'quantidade': 1, 'val..."
1,102,1002,"[{'produto': 'Smartphone', 'quantidade': 1, 'v..."


O resultado não incluirá a expansão da lista em `compras`. Isso acontece porque `pd.json_normalize` expande somente estruturas do tipo dicionário. Quando encontra uma lista, ele não consegue continuar a transformação.

Para resolver esse problema, podemos usar o parâmetro `record_path`, especificando o nome da coluna que contém a lista a ser expandida.

**Atenção:** Ao usar apenas o parâmetro `record_path`, podemos perder as colunas de nível superior, como `id_vendedor` e `id_cliente`. Vamos verificar esse comportamento:

In [12]:
df = pd.json_normalize(dados, record_path='compras')
df

,produto,quantidade,valor_unitario,valor_total
0,Notebook,1,3500.0,3500.0
1,Mouse,2,50.0,100.0
2,Smartphone,1,1200.0,1200.0
3,Fone de Ouvido,1,200.0,200.0


As colunas `id_vendedor` e `id_cliente` desaparecem! Para preservar essas informações, adicionamos o parâmetro `meta`, que define os campos de nível superior a serem mantidos no DataFrame:

In [13]:
df = pd.json_normalize(dados, record_path='compras', meta = ['id_vendedor', 'id_cliente'])
df

,produto,quantidade,valor_unitario,valor_total,id_vendedor,id_cliente
0,Notebook,1,3500.0,3500.0,101,1001
1,Mouse,2,50.0,100.0,101,1001
2,Smartphone,1,1200.0,1200.0,102,1002
3,Fone de Ouvido,1,200.0,200.0,102,1002


E isso é tudo que você precisa saber para importar arquivos JSON, normalizar suas estruturas e utilizá-los em seus projetos de Data Science.

Agora é sua vez de praticar o conhecimento adquirido! Realize os exercícios e sinta-se à vontade para compartilhar o que aprendeu. Boas práticas!